## L-mode H-mode  Classifier

In [1]:
import os
from pathlib import Path
import matplotlib.pyplot as plt
import confinement_mode_classifier as cmc
from datetime import datetime
import time 
import torchvision
import torch
from torch.optim import lr_scheduler
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter

Seed set to 42


Device: cuda:0


In [2]:
path = Path(os.getcwd())
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [3]:
batch_size = 32
shots = [16534, 16769, 16773, 18130, 19237, 19240, 19379, 18057, 16989]
shots_for_testing = [16769, 18130, 18057]
shots_for_validation = [19237]

shot_df, test_df, val_df, train_df = cmc.load_and_split_dataframes(path,shots, shots_for_testing, shots_for_validation )


test_dataloader = cmc.get_dloader(test_df, path=path, batch_size=batch_size)
val_dataloader = cmc.get_dloader(val_df, path=path, batch_size=batch_size)
train_dataloader = cmc.get_dloader(train_df, path=path, batch_size=batch_size)

In [12]:
print(train_df)

        time mode                      filename   shot
0      960.2    0   imgs/RIS1_16534_t=960.2.png  16534
1      960.4    0   imgs/RIS1_16534_t=960.4.png  16534
2      960.6    0   imgs/RIS1_16534_t=960.6.png  16534
3      960.8    0   imgs/RIS1_16534_t=960.8.png  16534
4      961.0    0   imgs/RIS1_16534_t=961.0.png  16534
...      ...  ...                           ...    ...
8947  1097.4    1  imgs/RIS1_16989_t=1097.4.png  16989
8948  1097.6    1  imgs/RIS1_16989_t=1097.6.png  16989
8949  1097.8    1  imgs/RIS1_16989_t=1097.8.png  16989
8950  1098.0    1  imgs/RIS1_16989_t=1098.0.png  16989
8951  1098.2    1  imgs/RIS1_16989_t=1098.2.png  16989

[8952 rows x 4 columns]


### Create dataloader, that will be used in training

In [4]:
dataloaders = {'train':train_dataloader, 'val':val_dataloader}
dataset_sizes = {x: len(dataloaders[x].dataset) for x in ['train', 'val']}

In [5]:
timestamp =  datetime.fromtimestamp(time.time()).strftime("%d-%m-%Y, %H-%M-%S ") + input('add comment: ')

# create grid of images
# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter(f'runs/{timestamp}')

## Import ResNet pretrained model
### And freeze all layers except last f.c. layer


In [6]:
pretrained_model = torchvision.models.resnet18(weights='IMAGENET1K_V1', )
for param in pretrained_model.parameters():
    param.requires_grad = False
 
# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = pretrained_model.fc.in_features
pretrained_model.fc = nn.Linear(num_ftrs, 3) #3 classes: L-mode, H-mode, ELM
pretrained_model = pretrained_model.to(device)

### Hyperparameters

In [7]:
#
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = torch.optim.Adam(pretrained_model.parameters(), lr=0.001) #pouzit adam

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

## Train last fc of the model

In [8]:
num_epochs = 2
model = cmc.train_model(pretrained_model, criterion, optimizer, exp_lr_scheduler, 
                       dataloaders, writer, dataset_sizes, num_epochs=num_epochs, comment = 'fc training')
model_path = Path(f'{path}/runs/{timestamp}/model_fc_trained.pt')
torch.save(model.state_dict(), model_path)

Epoch 1/2
----------


  0%|          | 0/280 [00:00<?, ?it/s]

train Loss: 0.5943 Acc: 0.7868


  0%|          | 0/61 [00:00<?, ?it/s]

val Loss: 0.6841 Acc: 0.6445

Epoch 2/2
----------


  0%|          | 0/280 [00:00<?, ?it/s]

train Loss: 0.3553 Acc: 0.8810


  0%|          | 0/61 [00:00<?, ?it/s]

val Loss: 0.6111 Acc: 0.6747

Training complete in 5m 33s
Best val Acc: 0.674693


### Test model with last fc trained

In [11]:

predictions_df, fig_confusion_matrix, f1, precision, recall, accuracy, pr_curve, roc_curve = cmc.test_model(f'runs/{timestamp}', model, test_dataloader)
# writer.add_figure(f'Confusion matrix for the model with trained f.c. layer', fig_confusion_matrix)
# writer.add_scalar(f'Accuracy on test_dataset with trained f.c. layer', accuracy)
# writer.add_scalar(f'F1 metric on test_dataset with trained f.c. layer', f1)
# writer.add_scalar(f'Precision on test_dataset with trained f.c. layer', precision)
# writer.add_scalar(f'Recall on test_dataset with trained f.c. layer', recall)
# writer.close()

Processing batches:   0%|          | 0/197 [00:00<?, ?it/s]

In [12]:
predictions_df

,shot,prediction,label,time,confidence,L_logit,H_logit,ELM_logit
0,18057,1,1,1134.8,0.502590,-1.793138,0.649422,0.547117
1,16769,0,1,1169.6,0.721079,1.098766,-1.121856,-0.180403
2,18130,0,0,1073.6,0.725163,1.232137,-1.048278,-0.052474
3,16769,0,0,1187.6,0.716587,0.776189,-0.808095,-0.882390
4,16769,1,1,1101.6,0.704434,-1.351699,0.881314,-0.282235
...,...,...,...,...,...,...,...,...
6273,16769,1,1,1122.0,0.898612,-2.169930,1.766729,-0.605063
6274,18130,0,0,1230.8,0.980773,2.794104,-1.365405,-2.730157
6275,18057,2,1,1169.0,0.575480,-1.515989,0.178997,0.651806
6276,16769,0,2,1155.0,0.701854,0.855418,-0.635270,-0.756125


In [13]:
f1

tensor(0.5917)

In [14]:
precision

tensor(0.5574)

In [15]:
recall

tensor(0.5908)

In [16]:
accuracy

0.5917489646384199

## Train the whole model

In [17]:
for param in model.parameters():
    param.requires_grad = True
model = cmc.train_model(pretrained_model, criterion, optimizer, exp_lr_scheduler, 
                        dataloaders, writer, dataset_sizes, num_epochs=16, comment = 'fc training')
model_path = Path(f'{path}/runs/{timestamp}/model_fully_trained.pt')
torch.save(model.state_dict(), model_path)

Epoch 1/16
----------


  0%|          | 0/280 [00:00<?, ?it/s]

train Loss: 0.2475 Acc: 0.9115


  0%|          | 0/61 [00:00<?, ?it/s]

val Loss: 2.3960 Acc: 0.3817

Epoch 2/16
----------


...

val Loss: 1.5419 Acc: 0.7469

Epoch 16/16
----------


  0%|          | 0/280 [00:00<?, ?it/s]

train Loss: 0.0034 Acc: 0.9991


  0%|          | 0/61 [00:00<?, ?it/s]

val Loss: 1.5384 Acc: 0.7392

Training complete in 56m 15s
Best val Acc: 0.834529


### Test model with all layers trained

In [23]:
from importlib import reload
reload(cmc)
predictions_df, fig_confusion_matrix, f1, precision, recall, accuracy, pr_curve, roc_curve = cmc.test_model(f'runs/{timestamp}', model, test_dataloader)
# writer.add_figure(f'Confusion matrix for the model with trained f.c. layer', fig_confusion_matrix)
# writer.add_scalar(f'Accuracy on test_dataset with trained f.c. layer', accuracy)
# writer.add_scalar(f'F1 metric on test_dataset with trained f.c. layer', f1)
# writer.add_scalar(f'Precision on test_dataset with trained f.c. layer', precision)
# writer.add_scalar(f'Recall on test_dataset with trained f.c. layer', recall)
# writer.close()

Seed set to 42


Device: cuda:0


Processing batches:   0%|          | 0/197 [00:00<?, ?it/s]